<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=141892ec618ada2cf9f4aeb7d1c1379ad546ac834fee1120612e8c54aae3f044
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-28 10:41:10
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  9.00 L
Current:  1.37 C
-------------------
Today PnL: -35.56 K (-0.26%)
Current PnL: -20.24 L (-13.61%)
CY Booked + Current PnL: -11.20 L (-7.53%)
-------------------
Total profit:  2.91 L
Total loss:  -23.15 L
-------------------
Total Booked + Current PnL: 15.78 L (12.97%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.6%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.16 C
Est FTT PnL: 78.63 L (57.39%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.90,64.0,H-MC,14.85,164117.0,37917.0,8255.0,1.06,30.05,5.03,36.59,108.0,4.59,1.28,46.38,XY24,NTT,HEALTHCARE
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.48,112424.0,8112.0,8533.0,-0.43,7.78,7.59,15.96,9.0,0.95,0.88,25.99,X5K,ATH,METALS
4,APOLLOHOSP,8285.00,-13.97,63.0,H-LC,8.31,171831.0,15654.0,10447.0,0.22,10.02,6.08,16.71,22.0,1.50,1.34,29.21,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-13.25,46.0,M-LC,4.09,214584.0,13104.0,14120.0,0.24,6.50,6.58,13.51,60.0,0.93,1.68,19.92,XY25,NTT,BANKS
81,TTKPRESTIG,770.00,97.01,56.0,M-SC,8.42,84921.0,-15856.0,15948.0,1.27,-15.73,18.78,0.09,245.0,-0.99,0.66,11.57,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RAJOOENG,143.33,-32.41,39.0,H-SC,9.53,102560.0,60.0,40768.0,-0.84,0.06,39.75,39.83,114.0,0.0,0.8,1.54,AR,ATH,MISC


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.59,32.0,H-LC,13.16,195098.0,-21421.0,114327.0,-0.39,-9.89,58.60,42.91,7.0,-0.19,1.52,2.17,AR,NTT,ELECTRICAL
45,ITC,452.00,-39.19,37.0,H-LC,3.14,145873.0,-4059.0,18657.0,-0.64,-2.71,12.79,9.74,4.0,-0.22,1.14,3.36,X40,NTT,FMCG
84,UNITDSPR,1644.00,-15.51,37.0,H-LC,8.74,169990.0,-11769.0,47019.0,-1.15,-6.47,27.66,19.39,86.0,-0.25,1.33,1.15,X40N,NTT,BREWERIES
33,HCLTECH,1943.91,0.22,39.0,H-LC,10.36,177852.0,-12794.0,59314.0,-2.34,-6.71,33.35,24.40,8.0,-0.22,1.39,8.16,X40,ATH,IT
47,JIOFIN,387.00,-0.58,41.0,H-LC,15.44,219940.0,2534.0,50960.0,-0.49,1.17,23.17,24.61,48.0,0.05,1.72,56.62,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.2,-7.51,46.0,H-LC,3.12,183744.0,-16566.0,76548.0,-1.19,-8.27,41.66,29.94,16.0,-0.22,1.44,26.94,X200,ATH,IT
45,ITC,452.0,-39.19,37.0,H-LC,3.14,145873.0,-4059.0,18657.0,-0.64,-2.71,12.79,9.74,4.0,-0.22,1.14,3.36,X40,NTT,FMCG
55,NESTLEIND,1377.0,-11.91,52.0,H-LC,3.18,271159.0,5733.0,51059.0,-0.29,2.16,18.83,21.40,11.0,0.11,2.12,9.65,XY25,NTT,FMCG
20,CIPLA,1795.0,-19.09,59.0,H-LC,4.22,215844.0,11344.0,30067.0,-0.27,5.55,13.93,20.25,10.0,0.38,1.69,14.76,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.15,57.0,H-LC,6.76,171278.0,-30108.0,117719.0,1.51,-14.95,68.73,43.51,27.0,-0.26,1.34,19.23,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-17.46,57.0,H-SC,3.20,116325.0,-11613.0,58744.0,2.34,-9.08,50.50,36.84,143.0,-0.20,0.91,19.09,X40N,ATH,ELECTRICAL
28,EASEMYTRIP,26.40,-13.20,34.0,M-SC,29.05,83528.0,-93287.0,174081.0,1.90,-52.76,208.41,45.70,197.0,-0.54,0.65,1.90,XY24,NTT,TRAVEL
75,TANLA,1963.11,-27.72,51.0,H-SC,8.07,164965.0,-74450.0,347400.0,1.80,-31.10,210.59,114.01,133.0,-0.21,1.29,51.30,AR,ATH,IT
5,ASIANPAINT,4250.00,-12.15,57.0,H-LC,6.76,171278.0,-30108.0,117719.0,1.51,-14.95,68.73,43.51,27.0,-0.26,1.34,19.23,X40,BTT,PAINTS
74,SYMPHONY,1306.00,-41.90,31.0,M-SC,17.03,123664.0,-47477.0,47425.0,1.37,-27.74,38.35,-0.03,196.0,-1.00,0.97,1.37,OX40N,NTT,DURABLES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDIAMART,4850.92,-51.09,49.0,H-SC,12.42,136581.0,13245.0,120519.0,-2.92,10.74,88.24,108.45,119.0,0.11,1.07,35.49,AR,ATH,MISC
33,HCLTECH,1943.91,0.22,39.0,H-LC,10.36,177852.0,-12794.0,59314.0,-2.34,-6.71,33.35,24.40,8.0,-0.22,1.39,8.16,X40,ATH,IT
79,TITAGARH,1548.00,-8.89,49.0,H-SC,9.52,149380.0,-42635.0,121521.0,-1.81,-22.20,81.35,41.08,158.0,-0.35,1.17,26.43,XY24,NTT,ENGINEERING
61,REPCOHOME,880.00,-62.00,24.0,H-SC,7.20,140116.0,-64882.0,210118.0,-1.61,-31.65,149.96,70.85,134.0,-0.31,1.09,15.11,XY24,NTT,FINANCE
77,TATAMOTORS,1065.00,-22.82,48.0,M-LC,4.91,243573.0,-32950.0,143026.0,-1.56,-11.92,58.72,39.81,54.0,-0.23,1.90,16.73,XY24,NTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,97.01,56.0,M-SC,8.42,84921.0,-15856.0,15948.0,1.27,-15.73,18.78,0.09,245.0,-0.99,0.66,11.57,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,39.0,H-SC,15.17,114669.0,-27810.0,47335.0,-0.75,-19.52,41.28,13.70,163.0,-0.59,0.90,35.34,OX40N,NTT,BANKS
48,KANSAINER,340.0,-69.07,46.0,H-SC,3.38,213075.0,-56592.0,92922.0,1.14,-20.99,43.61,13.47,138.0,-0.61,1.66,7.77,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,109.82,41.0,M-SC,1.96,137603.0,-36956.0,36988.0,0.53,-21.17,26.88,0.02,221.0,-1.00,1.07,19.69,OX40N,NTT,PAINTS
19,CERA,9475.0,-21.00,38.0,H-SC,3.01,112707.0,-31096.0,57841.0,-0.81,-21.62,51.32,18.60,149.0,-0.54,0.88,24.38,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.44,51.0,H-LC,15.73,243126.0,-52808.0,146727.0,-1.27,-17.84,60.35,31.73,1.0,-0.36,1.90,3.80,X40,BTT,IT
43,INFY,2275.00,-17.44,53.0,H-LC,11.79,270487.0,8886.0,136731.0,-1.21,3.40,50.55,55.67,3.0,0.06,2.11,9.71,X40,BTT,IT
45,ITC,452.00,-39.19,37.0,H-LC,3.14,145873.0,-4059.0,18657.0,-0.64,-2.71,12.79,9.74,4.0,-0.22,1.14,3.36,X40,NTT,FMCG
87,VBL,671.64,-10.56,52.0,H-LC,8.52,271421.0,6381.0,89922.0,-0.80,2.41,33.13,36.33,5.0,0.07,2.12,16.03,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,32.0,H-LC,13.16,195098.0,-21421.0,114327.0,-0.39,-9.89,58.60,42.91,7.0,-0.19,1.52,2.17,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,14.90,64.0,H-MC,14.85,164117.0,37917.0,8255.0,1.06,30.05,5.03,36.59,108.0,4.59,1.28,46.38,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.44,74.0,H-LC,11.95,231202.0,46721.0,23652.0,0.04,25.33,10.23,38.14,38.0,1.98,1.81,38.62,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.90,64.0,H-MC,14.85,164117.0,37917.0,8255.0,1.06,30.05,5.03,36.59,108.0,4.59,1.28,46.38,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,8.41,57.0,H-MC,4.59,187120.0,22450.0,45526.0,0.33,13.63,24.33,41.28,89.0,0.49,1.46,22.70,X40N,ATH,AC
34,HEROMOTOCO,5949.07,-3.00,72.0,H-MC,14.96,179942.0,29029.0,28269.0,1.34,19.24,15.71,37.97,93.0,1.03,1.41,49.41,AR,ATH,AUTO
88,VOLTAS,1918.49,-3.04,56.0,H-MC,6.55,203475.0,11733.0,84300.0,0.68,6.12,41.43,50.08,99.0,0.14,1.59,13.33,XY25,ATH,AC
24,DABUR,735.00,-6.90,53.0,H-MC,3.84,202488.0,5460.0,84154.0,-0.61,2.77,41.56,45.49,102.0,0.06,1.58,16.17,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.00,72.0,H-MC,14.96,179942.0,29029.0,28269.0,1.34,19.24,15.71,37.97,93.0,1.03,1.41,49.41,AR,ATH,AUTO
40,INDIAMART,4850.92,-51.09,49.0,H-SC,12.42,136581.0,13245.0,120519.0,-2.92,10.74,88.24,108.45,119.0,0.11,1.07,35.49,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.68,46.0,M-SC,5.64,96975.0,5476.0,73274.0,0.62,5.99,75.56,86.07,223.0,0.07,0.76,40.28,XR,NTT,DURABLES
90,WIPRO,420.00,-11.98,51.0,M-LC,7.79,155000.0,4055.0,105400.0,-0.81,2.69,68.00,72.51,53.0,0.04,1.21,8.87,XR,NTT,IT
59,RAJOOENG,143.33,-32.41,39.0,H-SC,9.53,102560.0,60.0,40768.0,-0.84,0.06,39.75,39.83,114.0,0.00,0.80,1.54,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,66.0,L-SC,28.19,83251.0,-30298.0,70347.0,0.93,-26.68,84.50,35.27,268.0,-0.43,0.65,103.76,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.18,39.0,H-SC,16.26,88206.0,-12757.0,107285.0,-0.79,-12.64,121.63,93.63,148.0,-0.12,0.69,31.38,SR,ATH,CHEMICALS
52,MASFIN,398.61,-19.15,46.0,H-SC,12.61,91650.0,-6330.0,27935.0,-0.60,-6.46,30.48,22.05,152.0,-0.23,0.72,33.22,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.48,112424.0,8112.0,8533.0,-0.43,7.78,7.59,15.96,9.0,0.95,0.88,25.99,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,39.0,H-SC,15.17,114669.0,-27810.0,47335.0,-0.75,-19.52,41.28,13.70,163.0,-0.59,0.90,35.34,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,66.0,L-SC,28.19,83251.0,-30298.0,70347.0,0.93,-26.68,84.50,35.27,268.0,-0.43,0.65,103.76,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-3.00,72.0,H-MC,14.96,179942.0,29029.0,28269.0,1.34,19.24,15.71,37.97,93.0,1.03,1.41,49.41,AR,ATH,AUTO
27,DMART,5201.00,-7.44,74.0,H-LC,11.95,231202.0,46721.0,23652.0,0.04,25.33,10.23,38.14,38.0,1.98,1.81,38.62,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,8.41,57.0,H-MC,4.59,187120.0,22450.0,45526.0,0.33,13.63,24.33,41.28,89.0,0.49,1.46,22.70,X40N,ATH,AC
67,SFL,1287.00,39.23,61.0,M-SC,9.96,183786.0,-78432.0,148260.0,-1.16,-29.91,80.67,26.63,229.0,-0.53,1.44,14.27,XY24,NTT,MISC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.90
1,25,41.99
2,50,72.20


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.33
LC    34.52
MC    27.14
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.19
X40N     14.41
X40      14.14
XY25     10.04
XR        9.26
AR        8.73
OX40N     7.59
X5K       2.32
MH        1.74
X200      1.44
SR        1.13
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.07
H-MC    24.17
H-SC    23.12
M-SC    13.61
M-LC     7.37
M-MC     2.64
L-SC     1.60
L-LC     1.08
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.92,-1.66,35.31
IT,12.39,-17.63,76.37
BANKS,7.65,-16.37,65.78
FINANCE,7.49,-25.86,72.27
MISC,6.08,-18.23,86.00
PAINTS,5.73,-18.80,42.21
ELECTRICAL,4.88,-12.50,61.22
AUTO,4.72,-11.98,60.69
AC,3.81,0.72,32.91


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2897780.0,23
XR,1107597.0,13
AR,1090613.0,10
X40,791239.0,10
X40N,523693.0,11
OX40N,484725.0,10
XY25,457431.0,7
SR,198961.0,2
X5K,141460.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2814337.0,27
M-SC,1503448.0,17
H-MC,1282654.0,18
H-LC,1126376.0,17
M-LC,434470.0,5
M-MC,339882.0,2
L-SC,256272.0,3
L-MC,59716.0,1
L-LC,45714.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      951663.0      6
M-SC       XY24      879336.0      7
H-SC       AR        665358.0      5
           XR        532708.0      6
H-MC       XY24      493916.0      5
H-LC       X40       479148.0      5
M-MC       XY24      339882.0      2
H-LC       AR        237256.0      2
H-MC       X40       226981.0      4
H-SC       OX40N     210897.0      4
H-LC       X40N      201107.0      5
H-MC       XY25      200358.0      2
H-SC       SR        198961.0      2
           X40N      194065.0      3
M-LC       XY24      182023.0      2
L-SC       XR        165686.0      2
M-SC       AR        159730.0      2
M-LC       X5K       132927.0      1
M-SC       OX40N     131048.0      4
H-MC       X40N      128521.0      3
M-SC       XY25      124415.0      1
           XR        123809.0      2
H-MC       XR        120278.0      1
M-LC       XR        105400.0      1
L-SC       OX40N      90586.0      1
M-SC       X40        85110.0      1
H-LC       X200       76548.0      1
           XY25       72824.0      2
H-SC       MH         60685.0      1
L-MC       XR         59716.0      1
H-MC       OX40N      52194.0      1
H-LC       XY24       50960.0      1
L-LC       XY25       45714.0      1
H-MC       MH         32137.0      1
           AR         28269.0      1
M-LC       XY25       14120.0      1
H-LC       X5K         8533.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 46.0 seconds
